In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration, T5Tokenizer
import csv
import pandas as pd

import urllib.request,sys,time
from bs4 import BeautifulSoup, UnicodeDammit
import requests
from matplotlib import pyplot as plt
import os

from wordcloud import WordCloud
import pdfplumber
import re
from scipy.special import softmax
import numpy as np
import base64
import io, json

from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

: 

In [ ]:
## CNA sample article urls
# url = "https://www.channelnewsasia.com/sustainability/scientists-discover-how-air-pollution-triggers-lung-cancer-2931111"
# url = "https://www.channelnewsasia.com/asia/indonesia-subsidised-fuel-price-increase-micro-businesses-2929006"
url = "https://www.channelnewsasia.com/singapore/nightclub-operator-club-posh-west-palace-entertainment-evade-gst-money-laundering-jailed-penalty-2935436"
# url = "https://cnaluxury.channelnewsasia.com/people/chye-seng-huat-hardware-leon-foo-morning-coffee-machine-206531"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM

def generate_sentiments(messageJson):

    # sentiment analysis model
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
    model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
    tokenizer.save_pretrained("./models/roberta")
    model.save_pretrained("./models/roberta")

### TEST CODE
requestJson = {"text": 'text'}
sentimentJson = generate_sentiments(requestJson)

In [ ]:
## convert CNA url -> HTML -> CSV
def extract_csv_from_html(url):
    try:
        page=requests.get(url) 
    except Exception as e:    
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                     
        print (error_type, 'Line:', error_info.tb_lineno)

    time.sleep(2)
    soup = BeautifulSoup(page.text, "html.parser")


    ## extract article content
    textContent = soup.find_all('div', attrs={'class':'text'})
    paragraphs = []

    for i in textContent:
        para = i.find_all('p')
        for j in para:
            content = j.getText().strip()
            paragraphs.append(content)

    df = pd.DataFrame(paragraphs)
    df.to_csv("backend/data/news.csv", index=False)


message = extract_csv_from_html(url)

In [ ]:
## convert cna url -> html -> txt
def extract_txt_from_html(url):
    try:
        page=requests.get(url) 
    except Exception as e:    
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                     
        print (error_type, 'Line:', error_info.tb_lineno)

    time.sleep(2)
    soup = BeautifulSoup(page.text, "html.parser")

    ## extract article title
    title = soup.find('h1',attrs={'class':'h1 h1--page-title'}).getText()

    ## extract article content
    textContent = soup.find_all('div', attrs={'class':'text'})
    paragraphs = []

    for i in textContent:
        para = i.find_all('p')
        for j in para:
            content = j.getText().strip()
            paragraphs.append(content)

    single_para = "".join(paragraphs)
    # lines = "\n".join(paragraphs)
    # multi_para = "\n".join(paragraphs)

    f = open("backend/data/article.txt", "w+")
    f.write(single_para)
    f.close()

message = extract_txt_from_html(url)

Text Pre-processing

In [2]:
# extracts content from csv/txt in /data to an entire paragraph
def extract_text(filename):
    message = ""
    name, extension = os.path.splitext(filename)
    try: 
        if extension == ".csv":   # csv format
            with open("data/{}".format(filename), 'r', encoding="utf8") as f:
                reader = csv.reader(f)
                next(reader, None)  # skip headers
                for row in reader:
                    # print(row)    
                    message += "".join(row)
                # print(sentences)
        elif extension == ".txt":     # txt format
            with open("data/{}".format(filename), 'rb') as f:
                byteString = f.read()
                dammit = UnicodeDammit(byteString, ["utf-8",  "ascii"])
                encoding = dammit.original_encoding
                print("Encoding Type:", encoding)
                message = byteString.decode(encoding, errors='ignore')
            
        elif extension == '.pdf':
            with pdfplumber.open("data/{}".format(filename)) as pdf:
                for page in pdf.pages:
                    message += page.extract_text()        
    except Exception as err:
            print(err, "occured in"+filename)

    dir_path = os.path.abspath('')
    nltk.data.path.append(dir_path + '/models/nltk_data')
    print("sentences: ", len(sent_tokenize(message))) 
    print("words: ", len(word_tokenize(message))) 
    
    return message

### LATEST NLP METHODS

In [7]:
## GENERATE SUMMARY: if text exceeds 512 tokens, it is cut off
def generate_summary(messageJson):
    # Summarization model
    tokenizer = AutoTokenizer.from_pretrained("models/bart-summary")
    model = AutoModelForSeq2SeqLM.from_pretrained("models/bart-summary")

    text = messageJson["text"]

    # encode text into tensor of integers
    inputs = tokenizer.encode("summarize:" + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
                    inputs, 
                    max_length=150, 
                    min_length=40, 
                    length_penalty=2.0, 
                    num_beams=4, 
                    early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(summary)
    
    returnJson = {"summary": summary}
    return returnJson


test_filename = "article3.txt"
text = extract_text(test_filename)
requestJson = {"text": text}
summarizedJson = generate_summary(requestJson)

Encoding Type: utf-8
Researchers from the National University of Singapore observed the data of 4,275 women with a history of gestational diabetes mellitus. Of these, 924 women had developed type 2 diabetes over a follow-up period of 28 years. A normal body mass index of 18.5 to 24.9, a high-quality diet, regular exercise, abstinence from smoking and moderate alcohol consumption were considered modifiable risk factors.


In [ ]:
# NLTK Sentence and Word Split
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize

def generate_sentiments(messageJson):
    dir_path = os.path.abspath('')
    nltk.data.path.append(dir_path + '/models/nltk_data')

    # sentiment analysis model
    tokenizer = AutoTokenizer.from_pretrained("models/roberta-SA")
    model = AutoModelForSequenceClassification.from_pretrained("models/roberta-SA")
    model.save("./roberta")
    labels=['Negative', 'Neutral', 'Positive']

    message = messageJson["text"]
    sentences = message.split('.')
    nltk_sentences = sent_tokenize(message)
    nltk_words = word_tokenize(message)
    print("NLTK sentence count:", len(nltk_sentences))
    print(message.split(' '))
    print("NLTK word count:", len(nltk_words))
    print(nltk_words)

### TEST CODE
test_filename = "article2.txt"
text = extract_text(test_filename)
num_words = len(text.split(' '))
num_sent = len(text.split('.'))
print("Total wordcount:", num_words)
print("Total sentences:", num_sent)
requestJson = {"text": text}
sentimentJson = generate_sentiments(requestJson)

In [48]:
# KEYBERT ALTERNATIVE
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import nltk, random
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer


def generate_topics(document, no_topics=3, no_top_words=10):

    modelPath = dir_path + '/models/all-MiniLM-L6-v2' 
    sentence_model = SentenceTransformer(modelPath)
    kw_model = KeyBERT(model=sentence_model)

    topics_words = kw_model.extract_keywords(document,
                                    # keyphrase_ngram_range=(1, 2),
                                    use_mmr=True,
                                    diversity=0.3,
                                    stop_words="english",
                                    # vectorizer=KeyphraseCountVectorizer(),
                                    vectorizer=KeyphraseCountVectorizer(pos_pattern='<N.*>'),
                                    top_n=8)
    print(topics_words)

    # Altering data structure to pass to frontend 
    outJson = {
        'name': 'Topics',
        'color': "hsl(13, 70%, 50%)",
        'children': []
    }

    t = 0
    for word in topics_words:
        t = t + 1
        word_vis = {
            "id": word[0],
            "value": round(word[1],3)
        }
        outJson['children'].append(word_vis)
    with open(dir_path +  '/output/topics.json', 'w', encoding='utf-8') as f:
            json.dump(outJson, f, ensure_ascii=False, indent=4) 
    return outJson

### TEST CODE    
dir_path = os.path.abspath('')
nltk.data.path.append(dir_path + '/models/nltk_data')

# f = open(dir_path + "/output/results.json")
# data = json.load(f)
# corpus = [doc["text"] for doc in data]

test_filename = "article3.txt"
document = extract_text(test_filename)
# data = [json["text"]]
# data = sent_tokenize(document)
generate_topics(document)

Encoding Type: utf-8
sentences:  8
words:  407
[('diabetes', 0.5276), ('obesity', 0.4081), ('diet', 0.3756), ('pregnancy', 0.291), ('health', 0.2517), ('prevalence', 0.236), ('risk', 0.1615), ('gdm', 0.1435)]


{'name': 'Topics',
 'color': 'hsl(13, 70%, 50%)',
 'children': [{'id': 'diabetes', 'value': 0.528},
  {'id': 'obesity', 'value': 0.408},
  {'id': 'diet', 'value': 0.376},
  {'id': 'pregnancy', 'value': 0.291},
  {'id': 'health', 'value': 0.252},
  {'id': 'prevalence', 'value': 0.236},
  {'id': 'risk', 'value': 0.162},
  {'id': 'gdm', 'value': 0.143}]}

In [16]:
# RUN BERTOPIC FIRST FOLLOWED BY KEYBERT
# BERTOPIC RUNS RANDOMLY UPON MULTIPLE TRIALS, UNLESS U SET UMAP 
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize
import nltk, random
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer



def generate_topics(document, no_topics=3, no_top_words=10):

    failBert = False
    global topics_words

    modelPath = dir_path + '/models/all-MiniLM-L6-v2' 
    sentence_model = SentenceTransformer(modelPath)
    vectorizer_model = CountVectorizer(stop_words="english")

    # OPTIMALLY RUN BERTOPIC
    if (failBert == False):
        try:
            data = sent_tokenize(document)
            topic_model = BERTopic(calculate_probabilities=True,
                                diversity=0.2,
                                embedding_model=sentence_model,
                                vectorizer_model=vectorizer_model,
                                verbose=True)

            topics, probabilities = topic_model.fit_transform(data)
            topics_words = topic_model.get_topic(0)
            print("BerTopic:")
            if (topics_words == False):
                failBert = True
        except Exception as e:
            failBert = True
            print(e) 
        
    if (failBert):
        # KEYBERT ALTERNATIVE FOR SHORTER DOCUMENTS
        kw_model = KeyBERT(model=sentence_model)

        topics_words = kw_model.extract_keywords(document,
                                        # keyphrase_ngram_range=(1, 2),
                                        use_mmr=True,
                                        diversity=0.3,
                                        stop_words="english",
                                        # vectorizer=KeyphraseCountVectorizer(),
                                        vectorizer=KeyphraseCountVectorizer(pos_pattern='<N.*>'),
                                        top_n=8)
        print("Keybert:")

    # Altering data structure to pass to frontend 
    outJson = []

    for word in topics_words:
        word_vis = {
            "id": word[0],
            "value": round(word[1],3)
        }
        outJson.append(word_vis)

    with open(dir_path +  '/output/topics.json', 'w', encoding='utf-8') as f:
            json.dump(outJson, f, ensure_ascii=False, indent=4) 
    return outJson



### TEST CODE    
dir_path = os.path.abspath('')
nltk.data.path.append(dir_path + '/models/nltk_data')

test_filename = "article_long.pdf"
document = extract_text(test_filename)
topics = generate_topics(document)
print(topics)


sentences:  47
words:  1412


Batches: 100%|██████████| 2/2 [00:00<00:00,  2.46it/s]
2022-10-07 09:54:07,842 - BERTopic - Transformed documents to Embeddings
2022-10-07 09:54:09,446 - BERTopic - Reduced dimensionality
2022-10-07 09:54:09,451 - BERTopic - Clustered reduced embeddings


BerTopic:
Keybert:
[{'id': 'hoarding', 'value': 0.597}, {'id': 'hoarders', 'value': 0.581}, {'id': 'hoarder', 'value': 0.52}, {'id': 'decluttering', 'value': 0.507}, {'id': 'containers', 'value': 0.303}, {'id': 'debris', 'value': 0.293}, {'id': 'permission', 'value': 0.182}, {'id': 'singapore', 'value': 0.181}]


In [12]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
import umap, hdbscan

from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer


### TEST CODE
    
dir_path = os.path.abspath('')
nltk.data.path.append(dir_path + '/models/nltk_data')

test_filename = "article_long.txt"
document = extract_text(test_filename)
# data = [json["text"]]
data = sent_tokenize(document)

# ---------------------- #

vectorizer_model = CountVectorizer(stop_words="english")

modelPath = dir_path + '/models/all-MiniLM-L6-v2' 
sentence_model = SentenceTransformer(modelPath)
# embeddings = sentence_model.encode(data, show_progress_bar=False)

# Create BERTopic model
topic_model = BERTopic(calculate_probabilities=True,
                    diversity=0.2,
                    embedding_model=sentence_model,
                    vectorizer_model=vectorizer_model,
                    verbose=True)

topics, probabilities = topic_model.fit_transform(data)
topic_info = topic_model.get_topic(0)
print(topic_info)
topic_model.visualize_barchart()


Encoding Type: utf-8
sentences:  66
words:  1551


Batches: 100%|██████████| 3/3 [00:01<00:00,  2.93it/s]
2022-10-06 11:18:20,241 - BERTopic - Transformed documents to Embeddings
2022-10-06 11:18:22,671 - BERTopic - Reduced dimensionality
2022-10-06 11:18:22,681 - BERTopic - Clustered reduced embeddings


[('hpb', 0.12494788212355505), ('salt', 0.09470908114110338), ('healthier', 0.08204789041552257), ('sodium', 0.0748586857336087), ('taste', 0.04864694923546847), ('alternatives', 0.04864694923546847), ('ingredient', 0.04496652293126789), ('chefs', 0.04496652293126789), ('lee', 0.0398366411605192), ('sauces', 0.03801727986976176)]


   Topic  Count                              Name
0     -1     57  -1_mental_health_services_people
1      0     32      0_mental_health_covid19_prof
2      1     23       1_mental_health_people_care
3      2     20    2_services_care_support_health

In [ ]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

def generate_topics(json, no_topics=3, no_top_words=10):
    dir_path = os.path.abspath('')
    nltk.data.path.append(dir_path + '/models/nltk_data')
    
    data = [json["text"]]
    stop_words = set(stopwords.words('english'))

    lemmatized_texts = lemmatization(data)
    print(lemmatized_texts)


### TEST CODE
test_filename = "covid.txt"
text = extract_text(test_filename)
requestJson = {"text": text}
sentimentJson = generate_topics(requestJson)

NLP Methods

In [ ]:
# summarizes entire paragraph and exports as JSON
def summarize(summarizer, chunks):
    result = ""
    for i in chunks:
        summarized = summarizer(i, max_length=70, min_length=30, do_sample=False)
        # print(summarized[0]["summary_text"])
        result += summarized[0]["summary_text"]
    # print(result)
    return result

def generate_summary(messageJson):
    # Summarization model
    model = "facebook/bart-large-cnn"
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForSeq2SeqLM.from_pretrained(model)
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
    message = messageJson["text"]
    sentences = message.split('.')
    chunks = form_text_chunks(sentences, 1024)
    # print("chunks:", chunks)
    summary = summarize(summarizer, chunks)
    # print(result + "\n")
    while (len(summary) > 1200):
        sentences = summary.split('.')
        chunks = form_text_chunks(sentences, 1024)
        summary = summarize(summarizer, chunks)
    print(summary)
    returnJson = {"summary": summary}
    # print("ReturnJSON:", returnJson)
    return returnJson


### TEST CODE
test_filename = "article.txt"
text = extract_text(test_filename)
num_words = len(text.split(' '))
print("Total wordcount:", num_words)
requestJson = {"text": text}
summarizedJson = generate_summary(requestJson)

In [ ]:
def generate_sentiments(message):

    nltk.data.path.append(dir_path + '/models/nltk_data')

    # sentiment analysis model
    tokenizer = AutoTokenizer.from_pretrained(dir_path + "/models/roberta-SA")
    model = AutoModelForSequenceClassification.from_pretrained(dir_path + "/models/roberta-SA")
    
    labels=['Negative', 'Neutral', 'Positive']

    sentences = sent_tokenize(message)
    
    scores_table = []
    sentimentJson = {"overall_score":{}, "overall_sentiment": "none", "sentiment_count":{}}
    sentiment_count = {"Negative": 0, "Neutral": 0, "Positive": 0}
    sentiment_by_sent = {}
    
    for index in range(len(sentences)):
        text = sentences[index]
        encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        output = model(**encoded_input)
        scores = output[0][0].detach().tolist()
        scores = softmax(scores)    # converts into probabilities

        best_score = np.round(np.max(scores), 4)
        scores_table.append(scores)
        pred_sentiment = labels[np.argmax(scores)]
        sentiment_by_sent[index] = {"text":text, "score":best_score, "sentiment":pred_sentiment}
        sentiment_count[pred_sentiment] += 1
    
    # calculate overall average sentiment       
    np_scores = np.asarray(scores_table)
    # print((np_scores))
    avg_sentiments = np.round(np.average(np_scores, axis=0), 4)
    for index in range(len(avg_sentiments)):
        sentiment = labels[index]
        sentimentJson["overall_score"][sentiment] = avg_sentiments[index]
    sentimentJson["overall_sentiment"] = labels[np.argmax(avg_sentiments)]
    sentimentJson["sentiment_count"] = sentiment_count
    sentimentJson["sentiment_distribution"] = sentiment_by_sent
    print(sentimentJson)
    return sentimentJson

### TEST CODE
test_filename = "covid.pdf"
text = extract_text(test_filename)
sentimentJson = generate_sentiments(text)

In [ ]:
def generate_word_cloud(messageJson):
    message = messageJson["text"]

    wc = WordCloud(background_color="white", width=1600, height=800)
    wc.generate(message)
    imageRes = wc.to_image()
    
    # display wordcloud
    plt.figure()
    plt.imshow(wc)
    plt.axis("off")
    plt.show()
    
    # Convert to bytestring 
    file_object = io.BytesIO()
    imageRes.save(file_object, format='PNG')
    bytestring = base64.b64encode(file_object.getvalue())
    returnJson = {"data": bytestring.decode('utf-8')}

    with open("wordcloud.json", "w", encoding='utf-8') as f:
        json.dump(returnJson, f)

    print(returnJson)
    return returnJson
    
    return None


### TEST CODE
test_filename = "sample_news.csv"
text = extract_text(test_filename)
num_words = len(text.split(' '))
num_sent = len(text.split('.'))
print("Total wordcount:", num_words)
print("Total sentences:", num_sent)
requestJson = {"text": text}
wordcloudJson = generate_word_cloud(requestJson)

In [ ]:
import base64
with open("backend/data/dataurl.txt", "r", encoding='utf-8') as f:
    lines = f.readlines()
    paragraph = "".join(lines)
    index = paragraph.index(',')
    url = paragraph[index+1:]
    print(base64.b64decode(url))
    with open("backend/data/test.txt", "wb") as f:
        f.write(base64.b64decode(url))

In [ ]:
# extract text & export NLP results as JSON
def run_chatterbox(filename):
    text = extract_text(filename)
    num_words = len(text.split(' '))
    num_sent = len(text.split('.'))
    requestJson = {"text": text}

    print("Total wordcount:", num_words)
    print("Total sentences:", num_sent)
    
    # Sentiment
    sentimentJson = generate_sentiments(requestJson)
    # Summary
    summarizedJson = generate_summary(requestJson)
    # Word Cloud
    wordcloudJson = generate_word_cloud(requestJson)
    
    finalJson = {**sentimentJson, **summarizedJson, **wordcloudJson}
    
    return finalJson

response = run_chatterbox("article.txt")
print(response)

In [ ]:
# Export individual text, sentiment and score into JSON
def export_sentiments_to_JSON(chunks, model, tokenizer):
        scores_table = []
        new_df = pd.DataFrame(columns=["Content","Sentiment", "Score"])
        for index in range(len(chunks)):
                text = chunks[index]
                encoded_input = tokenizer(text, return_tensors='pt')
                output = model(**encoded_input)
                scores = output[0][0].detach().tolist()
                scores = softmax(scores)    # converts into probabilities
                
                scores_table.append(scores)
                max_index = np.argmax(scores)
                pred_sentiment = labels[max_index]
                pred_score = scores[pred_sentiment]

                # write text and predicted data into df
                new_df.at[index, "Sentiment"] = pred_sentiment
                new_df.at[index, "Score"] = pred_score
                new_df.at[index, "Content"] = "".join((chunks[index]))
        new_df.to_json("backend/data/results.json")

OLD SENTINENT ANALYSIS METHOD WITH PIPELINE

In [ ]:
def generate_sentiments(messageJson):
    # sentiment analysis model
    # model = "siebert/sentiment-roberta-large-english"
    nlp = pipeline("sentiment-analysis", model=model, tokenizer=model)

    message = messageJson["text"]
    texts = message.split('.')
        
    print(texts, "\n\n")
    chunks = form_text_chunks(texts, 512)
    new_df = pd.DataFrame(columns=["Content","Sentiment", "Score"])

    for index in range(len(chunks)):
        preds = nlp(chunks[index])
        # print(preds)
        pred_sentiment = preds[0]["label"]
        pred_score = preds[0]["score"]

        # write predicted data into df
        new_df.at[index, "Sentiment"] = pred_sentiment
        new_df.at[index, "Score"] = pred_score
        # write text
        new_df.at[index, "Content"] = "".join((chunks[index]))

    # new_df.to_csv("backend/data/results.csv", index=False)
    new_df.to_json("backend/data/results.json")
    returnJson = {"sentiment": new_df}
    print(returnJson)
    return returnJson
    return None

### TEST CODE
test_filename = "article.txt"
text = extract_text(test_filename)
num_words = len(text.split(' '))
num_sent = len(text.split('.'))
print("Total wordcount:", num_words)
print("Total sentences:", num_sent)
requestJson = {"text": text}
sentimentJson = generate_sentiments(requestJson)
# print(sentimentJson)

In [ ]:
def generate_sentiment_graph(results):    
    # visualize the sentiments
    sentiment_counts = results.groupby(['Sentiment']).size()
    print(sentiment_counts)
    fig = plt.figure(figsize=(6,6), dpi=100)
    ax = plt.subplot(111)
    sentiment_counts.plot.pie(ax=ax, autopct='%1.1f%%', startangle=270, fontsize=12, label="")

generate_sentiment_graph(results)